# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you learned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset
pokemon = pd.read_csv('./pokemon.csv')
pokemon.shape
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here

# What is the formula to compute Total?
# Total = sum of columns (HP, Attack, Defense, Sp. Atk, Sp. Def, Speed)

col_list= ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
pokemon['Total Test'] = pokemon[col_list].sum(axis=1)
pokemon

# Does the formula work for all pokemon? --> Yes!
pokemon['Total Test'].equals(pokemon['Total'])

True

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [49]:
# your code here

#categorical Variables
from sklearn.preprocessing import OneHotEncoder

X_cat = pokemon[['Type 1', 'Type 2']]
X_cat_type1 = pokemon[['Type 1']]
X_cat_type2 = pokemon[['Type 2']]

In [50]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
encoder = OneHotEncoder().fit(X_cat) # Do not drop the first column !!!
print(encoder.categories_)
encoded = encoder.transform(X_cat).toarray()

[array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object), array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water', nan], dtype=object)]


In [52]:
cols = encoder.get_feature_names_out(input_features=X_cat.columns)
cols

array(['Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric',
       'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying',
       'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground', 'Type 1_Ice',
       'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 'Type 1_Rock',
       'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark',
       'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy',
       'Type 2_Fighting', 'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost',
       'Type 2_Grass', 'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal',
       'Type 2_Poison', 'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel',
       'Type 2_Water', 'Type 2_nan'], dtype=object)

In [37]:
#cols = encoder.categories_[1]#, encoder.categories_[2] 
cols = encoder.get_feature_names_out(input_features=X_cat.columns)
cols

onehot_encoded = pd.DataFrame(encoded)
onehot_encoded = pd.DataFrame(encoded, columns=cols)
onehot_encoded.head()

,Type 1_Bug,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [41]:
# Divide into two dataframes
onehot_encoded_type1 = onehot_encoded.iloc[: , 0:18]
onehot_encoded_type2 = onehot_encoded.iloc[: , 19:]
onehot_encoded_type2

,Type 2_Dark,Type 2_Dragon,Type 2_Electric,Type 2_Fairy,Type 2_Fighting,Type 2_Fire,Type 2_Flying,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
798,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
res = []
for k, d in onehot_encoded.groupby(onehot_encoded.columns.str.split("_").str[0], axis=1):
    res.append(d)
    print(d)

# res(0)+res(1)

     Type 1_Bug  Type 1_Dark  Type 1_Dragon  Type 1_Electric  Type 1_Fairy  \
0           0.0          0.0            0.0              0.0           0.0   
1           0.0          0.0            0.0              0.0           0.0   
2           0.0          0.0            0.0              0.0           0.0   
3           0.0          0.0            0.0              0.0           0.0   
4           0.0          0.0            0.0              0.0           0.0   
..          ...          ...            ...              ...           ...   
795         0.0          0.0            0.0              0.0           0.0   
796         0.0          0.0            0.0              0.0           0.0   
797         0.0          0.0            0.0              0.0           0.0   
798         0.0          0.0            0.0              0.0           0.0   
799         0.0          0.0            0.0              0.0           0.0   

     Type 1_Fighting  Type 1_Fire  Type 1_Flying  Type 1_Ghost 

AttributeError: 'list' object has no attribute 'dtypes'

In [8]:
onehot_encoded.columns

Index(['Type 1_Bug', 'Type 1_Dark', 'Type 1_Dragon', 'Type 1_Electric',
       'Type 1_Fairy', 'Type 1_Fighting', 'Type 1_Fire', 'Type 1_Flying',
       'Type 1_Ghost', 'Type 1_Grass', 'Type 1_Ground', 'Type 1_Ice',
       'Type 1_Normal', 'Type 1_Poison', 'Type 1_Psychic', 'Type 1_Rock',
       'Type 1_Steel', 'Type 1_Water', 'Type 2_Bug', 'Type 2_Dark',
       'Type 2_Dragon', 'Type 2_Electric', 'Type 2_Fairy', 'Type 2_Fighting',
       'Type 2_Fire', 'Type 2_Flying', 'Type 2_Ghost', 'Type 2_Grass',
       'Type 2_Ground', 'Type 2_Ice', 'Type 2_Normal', 'Type 2_Poison',
       'Type 2_Psychic', 'Type 2_Rock', 'Type 2_Steel', 'Type 2_Water',
       'Type 2_nan'],
      dtype='object')

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [10]:
# your code here
y = pokemon[['Total']]
X_cat_encoded = onehot_encoded

y.corrwith(X_cat_encoded)

Collinearity = pd.concat((y, X_cat_encoded), axis=1)
corr_matrix_cat = round((Collinearity.corr()),2)
corr_matrix_cat

,Total,Type 1_Bug,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan
Total,1.00,-0.14,0.02,0.20,0.02,-0.03,-0.03,0.05,0.03,0.01,...,-0.04,0.02,0.10,-0.01,-0.07,0.08,-0.00,0.07,-0.02,-0.19
Type 1_Bug,-0.14,1.00,-0.06,-0.06,-0.07,-0.05,-0.06,-0.08,-0.02,-0.06,...,0.10,-0.02,-0.04,-0.02,0.20,-0.06,0.06,0.14,-0.01,-0.15
Type 1_Dark,0.02,-0.06,1.00,-0.04,-0.05,-0.03,-0.04,-0.05,-0.01,-0.04,...,-0.04,-0.04,0.07,-0.01,-0.04,0.02,-0.03,0.05,-0.03,-0.06
Type 1_Dragon,0.20,-0.06,-0.04,1.00,-0.05,-0.03,-0.04,-0.05,-0.01,-0.04,...,-0.04,0.11,0.12,-0.01,-0.04,0.09,-0.03,-0.03,-0.03,-0.06
Type 1_Electric,0.02,-0.07,-0.05,-0.05,1.00,-0.04,-0.05,-0.06,-0.02,-0.05,...,-0.01,-0.05,0.01,0.14,-0.05,-0.05,-0.03,0.06,0.01,0.06
Type 1_Fairy,-0.03,-0.05,-0.03,-0.03,-0.04,1.00,-0.03,-0.04,-0.01,-0.03,...,-0.03,-0.03,-0.02,-0.01,-0.03,-0.03,-0.02,-0.02,-0.02,0.12
Type 1_Fighting,-0.03,-0.06,-0.04,-0.04,-0.05,-0.03,1.00,-0.05,-0.01,-0.04,...,-0.03,-0.04,-0.02,-0.01,-0.04,0.07,-0.02,0.05,-0.02,0.10
Type 1_Fire,0.05,-0.08,-0.05,-0.05,-0.06,-0.04,-0.05,1.00,-0.02,-0.05,...,-0.05,0.02,-0.04,0.13,-0.06,-0.00,0.00,-0.01,0.00,0.03
Type 1_Flying,0.03,-0.02,-0.01,-0.01,-0.02,-0.01,-0.01,-0.02,1.00,-0.01,...,-0.01,-0.02,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01,0.00
Type 1_Ghost,0.01,-0.06,-0.04,-0.04,-0.05,-0.03,-0.04,-0.05,-0.01,1.00,...,0.33,-0.04,-0.03,-0.01,0.08,-0.04,-0.03,-0.03,-0.03,-0.07


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here